In [10]:
import os
import re
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.image import imread
from scipy.fft import fft, ifft
import numpy as np
from numpy import diff
import cmath
import cv2 as cv
import skimage
from skimage.io import imread, imshow
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
from skimage import color, exposure, transform
from skimage.exposure import equalize_hist
from skimage.filters import difference_of_gaussians, window, butterworth
from scipy.fft import fftn, fftshift
import time
from scipy.signal import argrelextrema
from natsort import natsorted, ns
from tifffile import imwrite
from PIL import Image
from skimage.morphology import reconstruction, skeletonize, medial_axis
from skimage.exposure import rescale_intensity
import numpy as np
import imageio as io
import matplotlib.image as mpimg
from matplotlib import cm
from skimage.io import imread, imshow#, imwrite
import cv2
from skimage.transform import hough_line, hough_line_peaks
from math import atan2, cos, sin, sqrt, pi

In [28]:
# ALL FUNCTIONS

# function returns an array of names files in a specific directory that will be analysed/altered
def toAnalyseArray(path, looking_for):
    images = Path(path)
    dir_list = os.listdir(images)

    to_analyse = []

    for item in dir_list:
        if bool(re.findall(looking_for, item)):
            to_analyse.append(item)

    # sorting into number order
    to_analyse = natsorted(to_analyse, key=lambda y: y.lower())
    return to_analyse




def saveImage(img, save_path):
    i = Image.fromarray(img) 
    i.save(save_path)




def loadImage(load_path):
  img = cv.imread(load_path)
  if img is None:
    print("Error: File not found")
    exit(0)
  return img




# rotating the image
def rotate(img, angle):
    height, width = img.shape
    center = (width/2, height/2)
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1)  # anticlockwise rotation
    print('matrix found')
    rotated_image = cv2.warpAffine(src=img, M=rotate_matrix, dsize=(width, height))
    print('rotated')
    #print(rotated_image.shape)
    return rotated_image
 

    
    
    
# https://automaticaddison.com/how-to-determine-the-orientation-of-an-object-using-opencv/
# don't really need to drawAxis but good for visualisation
def drawAxis(img, p_, q_, color, scale):
  p = list(p_)
  q = list(q_)
 
  ## [visualization1]
  angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
  hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
 
  # Here we lengthen the arrow by a factor of scale
  q[0] = p[0] - scale * hypotenuse * cos(angle)
  q[1] = p[1] - scale * hypotenuse * sin(angle)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
 
  # create the arrow hooks
  p[0] = q[0] + 9 * cos(angle + pi / 4)
  p[1] = q[1] + 9 * sin(angle + pi / 4)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
 
  p[0] = q[0] + 9 * cos(angle - pi / 4)
  p[1] = q[1] + 9 * sin(angle - pi / 4)
  cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
  ## [visualization1]




def getOrientation(pts, img):
  ## [pca]
  # Construct a buffer used by the pca analysis
  sz = len(pts)
  data_pts = np.empty((sz, 2), dtype=np.float64)
  for i in range(data_pts.shape[0]):
    data_pts[i,0] = pts[i,0,0]
    data_pts[i,1] = pts[i,0,1]
 
  # Perform PCA analysis
  mean = np.empty((0))
  mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)
 
  # Store the center of the object
  cntr = (int(mean[0,0]), int(mean[0,1]))
  ## [pca]
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
  return angle

"""   ## [visualization]
  # Draw the principal components
  cv.circle(img, cntr, 3, (255, 0, 255), 2)
  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
  drawAxis(img, cntr, p1, (255, 255, 0), 1)
  drawAxis(img, cntr, p2, (0, 0, 255), 5)
 
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
  ## [visualization]
 
  # Label with the rotation angle
  label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
  textbox = cv.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
  cv.putText(img, label, (cntr[0], cntr[1]), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA)
  """
  
 
 
 
 
 # find contours/angles of a single image
def findAngles(img):
  global all_angles,x,y,w,h
  
  # binarise the image
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  _, bw = cv.threshold(gray, 50, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)

  #imshow(bw)

  # Find all the contours in the thresholded image
  contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
  
  for i, c in enumerate(contours):
  
    # Calculate the area of each contour
    area = cv.contourArea(c)
    
    # Ignore contours that are too small (or too large -> would include 'or 5000000 < area:')
    if area < 3700: 
      continue
  
    # Draw each contour only for visualisation purposes
    #cv.drawContours(img, contours, i, (0, 0, 255), 2)
  
    # Find the orientation of each shape
    getOrientation(c, img)
    #print(getOrientation(c,img))
    all_angles[area] = getOrientation(c,img)  # area corresponding to the angle, fill this in for every image --> later you want 
  # to pick the angle corresponding to the largest area and then rotate all the images by that angle
  
  return img
  
  
  
  
# for one slice, we have added all the rows with OR
def findDimensionsOr(img):
    rowOr = np.zeros(len(img[0]))
    for row in img:
        rowOr = np.logical_or(rowOr, row)
    return(rowOr)




In [52]:
# CLEANING IMAGES AND FINDING ANGLES


# load all images in directory
#path = r'C:\\Adaptix\\lamb_femur\\recon\\'
# path = r'C:\\Adaptix\\2024-07-08\\BEA_3Dpart\\Recon\\BEA_3Dpart_25kV160uA1000ms10mm_recon\\'
path = r'C:\\Adaptix\\2024-07-08\\Plants\\Recon\\Maize2_35kV\\'
looking_for = '0.tif'
to_analyse = toAnalyseArray(path, looking_for)


all_angles = {}


for item in to_analyse:
    imgpath = path + str(item)
    img = loadImage(imgpath)
    img = findAngles(img)
    image = img[:,:,0]>0
    print(image.shape)
    image = skimage.morphology.remove_small_objects(image, min_size = 8)
    save_path = path + 'Cleaned'+ str(item) 
    saveImage(image, save_path)

print(all_angles)
maxarea = max(all_angles.keys())
angle = all_angles[maxarea]*180/np.pi
print(angle)





(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
(1300, 2304)
{23315.0: 0.37946379198902425, 4117.0: 1.2749882103810004, 10070.5: 1.628918793955564, 396864.0: -0.3266116052234911, 3893.5: 1.760192462681739, 22862.0: 0.34327863269973224, 7783.5: 0.7603794025146766, 10839.5: 1.5729715894319263, 385720.0: -0.17766180216489003, 21543.5: 0.433053725889397, 11081.0: 1.6350655342597895, 9042.0: 1.0015796488531086, 4346.0: 0.35106466225274185, 380306.0: 1.9064526725311755, 3717.0: -0.24119947771664915, 8678.0: -0.5951753275142876, 8007.0: 1.1281030745032732, 10927.0: 1.6408748523089194, 13473.0: 1.7466848914128899, 375477.0: 1.2540879489345542, 8

In [54]:
# ROTATING IMAGES

# path = r'C:\\Adaptix\\2024-07-08\\BEA_3Dpart\\Recon\\BEA_3Dpart_25kV160uA1000ms10mm_recon\\'
# path = r'C:\\Adaptix\\lamb_femur\\recon\\'
path = r'C:\\Adaptix\\2024-07-08\\Plants\\Recon\\Maize2_35kV\\'
looking_for = 'Cleaned'
to_analyse = toAnalyseArray(path, looking_for)

# rotate all the images by a certain angle 
for item in to_analyse:
    imgpath = path + str(item)
    image = mpimg.imread(imgpath)[:,:,0]
    
    rotated = rotate(image, angle)
    print(rotated.shape)
    save_path = path + r'\\Rotated'+ item 
    i = Image.fromarray(rotated) 
    i.save(save_path)


matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)
matrix found
rotated
(1300, 2304)


In [55]:
# CROPPING IMAGES


# folder we want to be in
# path = r'C:\\Adaptix\\2024-03-11\\Recon\\DowtyBlade_SolidNew\\'
# path = r'C:\\Adaptix\\lamb_femur\\recon\\'
# path = r'C:\\Adaptix\\2024-07-08\\BEA_3Dpart\\Recon\\BEA_3Dpart_25kV160uA1000ms10mm_recon\\'
path = r'C:\\Adaptix\\2024-07-08\\Plants\\Recon\\Maize2_35kV\\'
lookingfor = 'RotatedCleaned'
to_analyse = toAnalyseArray(path, lookingfor)



added_rows_matx = []
added_cols_matx = []

for item in to_analyse: 
    imgpath = path + str(item)
    # Read Images
    img = mpimg.imread(imgpath) > 0
    print(img)
    # img = np.array(image)
    # print(img.shape)
    
    added_rows_matx.append(findDimensionsOr(img))  #[:,:,0]
    added_cols_matx.append(findDimensionsOr(img.T))  #[:,:,0]
# print(added_rows_all)
#added_rows_matx = np.array(added_rows_all)
#print(added_rows_all.shape)

added_rows_arry = findDimensionsOr(added_rows_matx)
added_cols_arry = findDimensionsOr(added_cols_matx)

# finding indicies of values in teh array which are nonzero
nonzero_idx_rows = np.array(np.nonzero(added_rows_arry))
nonzero_idx_cols = np.array(np.nonzero(added_cols_arry))



print(f'min/max pixels {nonzero_idx_rows.min()}, {nonzero_idx_rows.max()}; min/max measured vals horizontally {nonzero_idx_rows.min()*0.14}mm, {nonzero_idx_rows.max()*0.14}mm; object width {np.abs(nonzero_idx_rows.min()*0.14 - nonzero_idx_rows.max()*0.14)}mm')
print(f'min/max pixels {nonzero_idx_cols.min()}, {nonzero_idx_cols.max()}; min/max measured vals vertically {nonzero_idx_cols.min()*0.14}mm, {nonzero_idx_cols.max()*0.14}mm; object height {np.abs(nonzero_idx_cols.min()*0.14 - nonzero_idx_cols.max()*0.14)}mm')




# crop each slice using array slicing

for item in to_analyse: 
    imgpath = path + str(item)
    # print(item)
    # Read Images
    img = mpimg.imread(imgpath)#[:,:,0]
    img = np.array(img)
    
    
    # cropped = img[nonzero_idx_rows.min():nonzero_idx_rows.max(), nonzero_idx_cols.min():nonzero_idx_cols.max()] 
    cropped = img[int(nonzero_idx_cols.min()):int(nonzero_idx_cols.max()), int(nonzero_idx_rows.min()):int(nonzero_idx_rows.max())] 
    save_path = path + r'\\Cropped'+ item 
    i = Image.fromarray(cropped)  # 
    i.save(save_path)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False F

In [53]:
# deleting files

# folder we want to be in
#path = r'C:\\Users\\ArzinaIsmaili\\OneDrive - Adaptix Limited\\Desktop\\deconvolution_adaptix2024\\HeightRecogCode\\DowtyPrismBlade2ndHalf\\'
path = r'C:\\Adaptix\\2024-07-08\\Plants\\Recon\\Maize2_35kV\\'
filled_images = Path(path)
dir_list = os.listdir(filled_images)

prefix = 'edC'
to_delete = []

for item in dir_list:
    if bool(re.findall(prefix, item)):
        to_delete.append(item)

for item in to_delete:
    #print(item)
    os.remove(path+str(item))